<a href="https://colab.research.google.com/github/rostro36/Partisan-Responses/blob/master/09_GraphWriter_Generation_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Partisan-Responses-master')

Install all requirements.

In [ ]:
!pip install wandb neuralcoref allennlp hnswlib allennlp-models spacy==2.1.0 torchtext==0.6.0 torch==1.5.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html
!python -m spacy download en

Import everything

In [ ]:
import neuralcoref
import spacy
import re
import pickle
import pandas as pd
import gc
from Answer import Answer
import utils

Load corefferenced data
You can download it from [here](https://drive.google.com/file/d/1rCPn8OrPJsKtInuUlhuiRGoBOzthwapN/view?usp=sharing).

In [4]:
df = pd.read_pickle("answers.pkl")
df.head()

,question,party,speech
0,Should abortion be illegal?,R,Mr. Speaker. Congress now has an opportunity t...
1,Should abortion be illegal?,R,I rise today to express my opposition to both ...
2,Should abortion be illegal?,R,"I am glad to follow my colleague from Utah, I ..."
3,Should abortion be illegal?,D,I rise in reluctant support of the Greenwood A...
4,Should abortion be illegal?,D,"my lead cosponsor. Senator HATCH, will appear ..."


Setup for parsing

In [14]:
def parse_entry(entry,verb_dict,verb_list):
    result=dict()
    result['question']=' '.join([token.text for token in utils.sp(entry['question'])])
    phrase_corpus, triplet_id, parsed_text,parsed=Answer(entry['speech']).create_test(verb_dict,verb_list)
    result['corpus']=' ; '.join(phrase_corpus)
    result['tags']=' '.join(['<phrase>']*len(phrase_corpus))
    result['triplet_id']=' ; '.join([re.sub('\,','',str(x))[1:-1] for x in triplet_id])
    result['parsed_text']=parsed_text
    result['parsed']=' '.join([str(x) for x in parsed])
    return result

def load_pickles(identifier, checkpoint):
  if os.path.exists("verb_dict"+identifier+str(checkpoint)+".pickle"):
    with open('verb_dict'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_dict = pickle.load(handle)
    with open('verb_list'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      verb_list = pickle.load(handle)
    with open('result'+identifier+str(checkpoint)+'.pickle', 'rb') as handle:
      result = pickle.load(handle)
  else:
    print('File does not exist yet.')
    verb_dict=dict()
    verb_list=[]
    result=[]
  return verb_dict, verb_list, result

def dump_pickles(identifier, checkpoint, verb_dict, verb_list, result):
  with open("verb_dict"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("verb_list"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(verb_list, handle, protocol=pickle.HIGHEST_PROTOCOL)
  with open("result"+identifier+str(checkpoint)+".pickle", 'wb') as handle:
      pickle.dump(result, handle, protocol=pickle.HIGHEST_PROTOCOL)
  return 

verb_dict = pickle.load(open("verb_dict.pickle", "rb"))
verb_list = None

Process data

In [15]:
result = []
df = df[df['speech'].apply(lambda x: len(x)) < 1000]
df.apply(lambda x: result.append(parse_entry(x,verb_dict,verb_list)), axis=1)
dump_pickles(identifier,1+df.shape[0]/step_size,verb_dict,verb_list,result)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


DO
DO
DO
DO
DISFAVOR
NEE
DO
DO
DO
DO
DO
NEE
COOK


Post-processing of data

In [19]:
df=pd.DataFrame(result)
df=df[df.iloc[:,2].apply(lambda x: len(x)>1)]
df=df.dropna(axis=0, how='any')

Write to variables

In [20]:
identifier='../GraphWriter-master/data/preprocessed'
df.to_csv("/content/drive/My Drive/GraphWriter-master/data/preprocessed.test.tsv", sep='\t', index=False, header=False)

Use file with lowest vloss in the folder "full_speeches" of GraphWriter-master to generate responses on test data, you can download a weight from [here](https://drive.google.com/drive/folders/1oM4XnLVWPx_5pSdG9VneuppQ7rI802sh?usp=sharing) and also some data including a "relations.vocab" from [here](https://drive.google.com/drive/folders/1UdYt6teIMhx6OaIrFyhZCwQIHU7c0YXg?usp=sharing).

You have to create an according file in the folder GraphWriter-master/outputs called "name of lowest vloss".inputs.beam_predictions.cmdline.

E.g. in the next cell "9.vloss-3.980539.lr-0.1" was used.
And /content/drive/My Drive/GraphWriter-master/outputs/9.vloss-3.980539.lr-0.1.inputs.beam_predictions.cmdline was created

In [21]:
os.chdir('/content/drive/My Drive/GraphWriter-master')
!python generator.py -save=full_speeches/12.vloss-4.299755.lr-0.1

2020-08-23 19:56:08.348633: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Loading Data from  data/preprocessed.train.tsv
building vocab
Vocab sizes:
src 4
ent 19591
nerd 4
rel 17893
out 11284
graph
0 25
/content/drive/My Drive/GraphWriter-master/models/newmodel.py:153: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  cx = torch.tensor(hx)
CONGRESS now has an opportunity to put AN END to one of the most barbaric acts known to mankind . <unk> , this barbaric act is perfectly legal in the United States of America , this barbaric act is called partialbirth abortion . Actually this barbaric act is a procedure in which a baby is partially delivered , THE DOCTOR actually 

In [28]:
with open("/content/drive/My Drive/GraphWriter-master/outputs/12.vloss-4.299755.lr-0.1.inputs.beam_predictions.cmdline") as f:
  gw_gen_answers = f.readlines()  

In [30]:
df['GW_ANS'] = gw_gen_answers

In [31]:
df.head(25)

,question,corpus,tags,triplet_id,parsed_text,parsed,GW_ANS
0,Should abortion be illegal ?,Congress ; an end ; the doctor ; a baby ; it i...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 76 1 ; 2 8056 3 ; 0 4 4 ; 5 2133 6 ; 5 4 7 ;...,<phrase_0> now has an opportunity to put <phra...,10 -1 0 1 11 10 -1 10 -1 2 3 12 10 -1 4 13 5 6...,"i rise in strong support of this legislation ,..."
2,What do you believe about tax increases ?,I ; the balance of my time ; at this bill ; an...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 145 1 ; 0 383 2 ; 0 8310 2 ; 3 97 2 ; 0 3471...,"<phrase_0> yield myself <phrase_1> , I would s...",0 1 7 2 8 9 6 -1 3 10 6 -1 4 11 12 13 5 14 15 ...,i rise in strong opposition to this amendment ...
3,Should same - sex marriage be legal ?,this bill ; any particular result or any group...,<phrase> <phrase> <phrase> <phrase>,0 1512 1 ; 2 145 3,I would like to clarify the actual wording of ...,0 1 5 4 -1 4 -1 4 -1 2 3 6 4 -1,i rise in strong support of the motion to inst...
4,Should we have higher taxes for higher incomes ?,"the Democrats ; the Tax Code ; share""-I ; that...",<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 82 1 ; 0 237 1 ; 2 15 3 ; 3 5158 1 ; 1 565 3...,"So when the Democrats , in 1993 , said . "" <ph...",9 -1 0 1 10 11 2 3 12 13 14 4 5 15 6 16 7 17 9...,"i rise in strong support of this amendment , i..."
5,Should we have higher taxes for higher incomes ?,we ; What,<phrase> <phrase>,0 4201 1,That s correct . <phrase_1> <phrase_0> are see...,2 -1 0 1 3 2 -1 2 -1,i rise in strong support of the person amendme...
6,Should we have universal healthcare ?,"I ; public healthcare a priority ; H. Res , 13...",<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 4 1 ; 0 386 2 ; 0 4918 3 ; 0 540 4 ; 5 5158 6,today <phrase_0> stand for a commitment to mak...,0 1 8 2 9 7 -1 7 -1 3 10 7 -1 4 11 7 -1 7 -1 5...,i rise in strong support of the amendment by s...
7,Do government regulations hinder free market c...,a middle class ; higher wage jobs ; nt ; a ret...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 5158 1 ; 0 1525 1 ; 2 5158 3 ; 4 337 5,"We want <phrase_0> to have <phrase_1> , to ear...",0 1 7 8 2 3 9 6 -1 4 5 10 6 -1 6 -1,i rise in strong support of the person amendme...
8,What do you think about the current president ?,you ; section 179 ; a small business ; a certa...,<phrase> <phrase> <phrase> <phrase>,0 1596 1 ; 2 9154 3,This is without a doubt . And I m glad <phrase...,4 -1 0 1 5 2 3 6 4 -1 4 -1,i rise in strong support of the person amendme...
9,What do you think about the current president ?,the courts ; the balance ; the purpose of the ...,<phrase> <phrase> <phrase> <phrase> <phrase> <...,0 252 1 ; 0 500 2 ; 3 7406 4 ; 3 1388 5 ; 3 12...,That is correct . What we are trying to do is ...,6 -1 0 1 7 2 8 3 4 9 5 10 11 6 -1,i would like to point out to the gentleman fro...
10,What do you think about the current president ?,I ; the Senator yielding ; a couple of questio...,<phrase> <phrase> <phrase> <phrase> <phrase>,0 2153 1 ; 0 7347 2 ; 3 1004 4,"<phrase_0> appreciate <phrase_1> , <phrase_0> ...",0 1 6 2 7 5 -1 3 4 8 5 -1 5 -1 5 -1,i rise in strong opposition to the pending ame...
